<a href="https://colab.research.google.com/github/tylerfeldman321/GANs-for-Medical-Data/blob/main/ECG_Synthesis_and_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline

In [ ]:
import os
if not os.path.exists('/content/drive/MyDrive/MUSER'):
    os.mkdir('/content/drive/MyDrive/MUSER')

%cd '/content/drive/MyDrive/MUSER'
if not os.path.exists('ECG-Synthesis-and-Classification'):
    !git clone https://github.com/tylerfeldman321/ECG-Synthesis-and-Classification.git

In [ ]:
%cd '/content/drive/MyDrive/MUSER/ECG-Synthesis-and-Classification/ecg_gan'
import pandas as pd

# train_path = 'mitbih_train.csv'
# test_path = 'mitbih_test.csv'
train_path = 
test_path = 

df_dataset = pd.read_csv(train_path)

In [ ]:
df_train = pd.read_csv(train_path, header=None)
df_test = pd.read_csv(test_path, header=None)
df_dataset = pd.concat([df_train, df_test], axis=0)
df_dataset.rename(columns={len(df_dataset.columns)-1: 'class'}, inplace=True)

id_to_label = {
    0: "Normal",
    1: "Artial Premature",
    2: "Premature ventricular contraction",
    3: "Fusion of ventricular and normal",
    4: "Fusion of paced and normal"
}
df_dataset['label'] = df_dataset.iloc[:, -1].map(id_to_label)
print(df_dataset.info())

In [ ]:
df_dataset.head()

In [ ]:
df_dataset.to_csv('data.csv', index=False)

In [ ]:
df_dataset['label'].value_counts()

In [ ]:
# Plot examples of the input data
import matplotlib.pyplot as plt
indexes = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
for index in indexes:
    row = df_dataset.iloc[index, :-2]
    row.plot(label=f'ECG Sample {index}')
    plt.xlabel('Time Index')
    plt.ylabel('Magnitude')
    plt.title('ECG Samples')

In [ ]:
!python train.py

In [ ]:
results_subdir = 

In [ ]:
# Plot generator and discriminator losses
import os
import pickle
results_dir = os.path.join('results', results_subdir)

with open(os.path.join(results_dir, 'Errors.pickle'), 'rb') as f:
    losses = pickle.load(f)
    print(losses['g'])
    print(losses['d'])
    epoch = np.arange(0, len(losses['g']), 1)
    plt.plot(epoch, losses['g'], label='Generator Loss')
    plt.plot(epoch, losses['d'], label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Generator and Discriminator Loss vs. Epoch')

In [ ]:
def mmd_rbf(X, Y, gamma=1.0):
    """MMD using rbf (gaussian) kernel (i.e., k(x,y) = exp(-gamma * ||x-y||^2 / 2))
    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]
    Keyword Arguments:
        gamma {float} -- [kernel parameter] (default: {1.0})
    Returns:
        [scalar] -- [MMD value]
    """
    XX = metrics.pairwise.rbf_kernel(X, X, gamma)
    YY = metrics.pairwise.rbf_kernel(Y, Y, gamma)
    XY = metrics.pairwise.rbf_kernel(X, Y, gamma)
    return XX.mean() + YY.mean() - 2 * XY.mean()

In [ ]:
# Plot generated samples for each epoch
import glob
import numpy as np

results_dir = os.path.join('results', results_subdir)

data_files = glob.glob(os.path.join(results_dir, '*.npy'))

plot_mmd = True
epochs = []
for data_file in data_files:
    epoch = str(os.path.basename(data_file).split('-')[1])
    with open(data_file, 'rb') as f:
        data = np.load(f)
        plt.plot(data)
        plt.xlabel('Time Index')
        plt.ylabel('Magnitude')
        title = 'Epoch' + ' ' + epoch
        plt.title(title)
        plt.show()

        if plot_mmd:
            mmd = mmd_rbf(test_data, data.T)
            print(f"Epoch: {epoch}, mmd: {mmd}")
            epochs.append(int(epoch))
            mmd_scores.append(mmd)

if plot_mmd:
    print(epochs)
    print(mmd_scores)
    plt.figure()
    plt.plot(epochs, mmd_scores, 'b-')
    plt.title('MMD Scores Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('MMD Score')
    plt.show()

    plt.figure()
    plt.plot(epochs, np.log(mmd_scores), 'b-')
    plt.title('MMD Scores Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('MMD Score')
    plt.show()